In [10]:
#@title Initialize and prepare Whisper environment

%pip install -q git+https://github.com/openai/whisper.git
%pip install -q python-docx
!git clone -b master https://github.com/bhnatt/whisper-am.git

#https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
fatal: destination path 'whisper-am' already exists and is not an empty directory.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
#@title input parameters
import sys
#sys.path.append('whisper-am')

from whisper_am import WhisperAM, mountDrive
from whisper_utils import write_txt, write_srt

data_dir = 'data' #@param {type:"string"}
if data_dir [-1] != '/' : data_dir = data_dir + '/'

ds = mountDrive ()
google_drive = '/content/drive/MyDrive/' if ds else './'
data_path   = google_drive + data_dir
model_name = 'tiny.en' #@param ["medium.en", "large", "tiny.en"]

am = WhisperAM (model_name, data_path)
if am.checkCuda () != 'cuda' :
    print ('Change colab runtime to GPU...')
    exit ()

Not a Google Colab environment !!!
Change colab runtime to GPU...


In [2]:
%%time
#@title run Whisper
import time

print ('Start :', time.strftime('%X %x %Z'))
am.run ()
print ('End  :', time.strftime('%X %x %Z'))

Start : 19:14:11 12/02/22 KST
./data/sample-0.mp3


/home/dwchoi/miniconda3/envs/stt/lib/python3.10/site-packages/whisper/transcribe.py:76: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 My thought I have nobody by a beauty and will as  ... s simple and devoted about to that might in a way.
End  : 19:15:21 12/02/22 KST
CPU times: user 1min 7s, sys: 1.79 s, total: 1min 9s
Wall time: 1min 9s


In [34]:
#https://www.programcreek.com/python/example/117479/ffmpeg.output

import ffmpeg
stream = ffmpeg.input ('./data/sample-0.mp3')
audio = stream.audio
stream = ffmpeg.output(audio, './data/sample-0-16k.mp3',
                       **{'ar': '16000', 'acodec': 'mp3'}).overwrite_output ()
out = ffmpeg.run (stream, capture_stdout=True, capture_stderr=True)